In [1]:
import xlrd
from pathlib import Path
import pandas as pd
import numpy as np
from numba import decorators
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# read in the data file
# Give the location of the file 

df = pd.read_excel(r'data/data.xlsx', sheet_name='totals')
# print(df)

In [4]:
## LOADING IN DATASETS

dataset = Path.cwd().joinpath("SongEmotionDataset")
datasheet = Path.cwd().joinpath("data") # for csua

#emotion labels
label_loc = datasheet.joinpath("data.xlsx")
wb = xlrd.open_workbook(label_loc) 
sheet = wb.sheet_by_index(1)

#emotion arr
emotions = ["amazement", "solemnity", "tenderness", "nostalgia", "calmness", "power", "joyful activation", "tension", "sadness"]

train_song = []
test_song = []
train_emotion = []
test_emotion = []

for i in range(1, 401):
    count_total = sheet.cell_value(i, 11)
    if i % 5 == 0:
        test_song.append(dataset.joinpath("{}.mp3".format(i)))
        emotion_arr = []
        for j in range(5):
            emotion_arr.append(sheet.cell_value(i, 2 + j))
        test_emotion.append(torch.tensor(emotion_arr, device=device).float())
    else:
        train_song.append(dataset.joinpath("{}.mp3".format(i)))
        emotion_arr = []
        for j in range(9):
            emotion_arr.append(sheet.cell_value(i, 2 + j))
        train_emotion.append(torch.tensor(emotion_arr, device=device))

print(len(train_song), len(test_song))
print(len(train_emotion), len(test_emotion))

320 80
320 80


In [5]:
# train_emotion

In [6]:
class SongEmotionDataset(Dataset):
    """
    Song Emotion Dataset. Uses librosa to process mp3 files.
    Takes first 20 seconds, and samples every 10 to get processed audio tensor.
    """

    def __init__(self, mp3, labels, transform=None):
        """
        Args:
            mp3: list of paths to mp3 files
            labels: list of labels
        """
        self.labels = labels
        self.mp3 = mp3
        self.cache = {}
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        if index not in self.cache.keys():
#             print("index of " + str(index) + " was cached!")
            data, rate = librosa.load(self.mp3[index], sr=16000, duration=10)
            mfccs = librosa.feature.mfcc(y=data, sr=rate, n_mfcc=40)
            assert rate == 16000
            sample_tensor = torch.tensor(mfccs, device=device).float()
            downsampled_tensor = sample_tensor[::10]
    #         print(mfccs.shape, data.shape)

            self.cache[index] = (downsampled_tensor, F.softmax(self.labels[index]))
#         else:
#             print("index was cached! index of " + str(index))
        return self.cache[index]

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, (2,2))
        self.conv2 = nn.Conv2d(32, 64, (2,2))
        self.pool1 = nn.MaxPool2d((2,2))
        self.drop1 = nn.Dropout(p=0.25)
        self.flat1 = nn.Flatten()
        self.dense1 = nn.Linear(9920, 128)
        self.drop2 = nn.Dropout(p=0.5)
        self.dense2 = nn.Linear(128, 9)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        
        x = self.pool1(x)
        x = self.drop1(x)
        x = self.flat1(x)
        x = self.dense1(x)
        x = F.relu(x)
        
        x = self.drop2(x)
        x = self.dense2(x)
#         x = F.relu(x)
        
        return F.log_softmax(x, dim = 1)

model = Net()
model.to(device)
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(2, 2), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.25, inplace=False)
  (flat1): Flatten()
  (dense1): Linear(in_features=9920, out_features=128, bias=True)
  (drop2): Dropout(p=0.5, inplace=False)
  (dense2): Linear(in_features=128, out_features=9, bias=True)
)


In [8]:
train_set = SongEmotionDataset(train_song, train_emotion)
test_set = SongEmotionDataset(test_song, test_emotion)
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True, **kwargs)

optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

Train set size: 320
Test set size: 80


In [9]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data.unsqueeze_(1)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
#         output = output.view(-1, len(emotions))
#         print(output.shape, target.shape)
#         print(output, target)
        loss = F.kl_div(output, target)
#         loss = F.cross_entropy(output, target)
#         loss = nn.CrossEntropyLoss(output, target)
#         loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
#         scheduler.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [10]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data.unsqueeze_(1)
        output = model(data)
#         print(output)
#         output = output.permute(1, 0, 2)
        pred = output.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.max(1)[1]).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [11]:
import warnings

log_interval = 5
warnings.filterwarnings("ignore")
for epoch in range(1, 100):
    print("training epoch " + str(epoch))
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
#     scheduler.step()
    train(model, epoch)
    scheduler.step()
    test(model, epoch)

training epoch 1
Train Epoch: 1 [0/320 (0%)]	Loss: 1.170977
Train Epoch: 1 [40/320 (12%)]	Loss: 0.162720
Train Epoch: 1 [80/320 (25%)]	Loss: 0.180911
Train Epoch: 1 [120/320 (38%)]	Loss: 0.156344
Train Epoch: 1 [160/320 (50%)]	Loss: 0.165938
Train Epoch: 1 [200/320 (62%)]	Loss: 0.154591
Train Epoch: 1 [240/320 (75%)]	Loss: 0.156051
Train Epoch: 1 [280/320 (88%)]	Loss: 0.167274

Test set: Accuracy: 0/80 (0%)

training epoch 2
Train Epoch: 2 [0/320 (0%)]	Loss: 0.164995
Train Epoch: 2 [40/320 (12%)]	Loss: 0.178693
Train Epoch: 2 [80/320 (25%)]	Loss: 0.135610
Train Epoch: 2 [120/320 (38%)]	Loss: 0.170699
Train Epoch: 2 [160/320 (50%)]	Loss: 0.180278
Train Epoch: 2 [200/320 (62%)]	Loss: 0.135244
Train Epoch: 2 [240/320 (75%)]	Loss: 0.161647
Train Epoch: 2 [280/320 (88%)]	Loss: 0.159351

Test set: Accuracy: 0/80 (0%)

training epoch 3
Train Epoch: 3 [0/320 (0%)]	Loss: 0.153170
Train Epoch: 3 [40/320 (12%)]	Loss: 0.157495
Train Epoch: 3 [80/320 (25%)]	Loss: 0.166979
Train Epoch: 3 [120/320 (3

Train Epoch: 20 [240/320 (75%)]	Loss: 0.170605
Train Epoch: 20 [280/320 (88%)]	Loss: 0.180366

Test set: Accuracy: 28/80 (35%)

training epoch 21
Train Epoch: 21 [0/320 (0%)]	Loss: 0.181189
Train Epoch: 21 [40/320 (12%)]	Loss: 0.134985
Train Epoch: 21 [80/320 (25%)]	Loss: 0.158835
Train Epoch: 21 [120/320 (38%)]	Loss: 0.122532
Train Epoch: 21 [160/320 (50%)]	Loss: 0.184107
Train Epoch: 21 [200/320 (62%)]	Loss: 0.138964
Train Epoch: 21 [240/320 (75%)]	Loss: 0.125018
Train Epoch: 21 [280/320 (88%)]	Loss: 0.140889

Test set: Accuracy: 28/80 (35%)

training epoch 22
Train Epoch: 22 [0/320 (0%)]	Loss: 0.141523
Train Epoch: 22 [40/320 (12%)]	Loss: 0.121902
Train Epoch: 22 [80/320 (25%)]	Loss: 0.142154
Train Epoch: 22 [120/320 (38%)]	Loss: 0.154641
Train Epoch: 22 [160/320 (50%)]	Loss: 0.178546
Train Epoch: 22 [200/320 (62%)]	Loss: 0.135306
Train Epoch: 22 [240/320 (75%)]	Loss: 0.142938
Train Epoch: 22 [280/320 (88%)]	Loss: 0.170047

Test set: Accuracy: 28/80 (35%)

training epoch 23
Train Ep

Train Epoch: 41 [0/320 (0%)]	Loss: 0.161095
Train Epoch: 41 [40/320 (12%)]	Loss: 0.163979
Train Epoch: 41 [80/320 (25%)]	Loss: 0.202407
Train Epoch: 41 [120/320 (38%)]	Loss: 0.140159
Train Epoch: 41 [160/320 (50%)]	Loss: 0.153754
Train Epoch: 41 [200/320 (62%)]	Loss: 0.159487
Train Epoch: 41 [240/320 (75%)]	Loss: 0.136649
Train Epoch: 41 [280/320 (88%)]	Loss: 0.138578

Test set: Accuracy: 28/80 (35%)

training epoch 42
Train Epoch: 42 [0/320 (0%)]	Loss: 0.164873
Train Epoch: 42 [40/320 (12%)]	Loss: 0.153355
Train Epoch: 42 [80/320 (25%)]	Loss: 0.157564
Train Epoch: 42 [120/320 (38%)]	Loss: 0.124845
Train Epoch: 42 [160/320 (50%)]	Loss: 0.133805
Train Epoch: 42 [200/320 (62%)]	Loss: 0.152376
Train Epoch: 42 [240/320 (75%)]	Loss: 0.143738
Train Epoch: 42 [280/320 (88%)]	Loss: 0.150798

Test set: Accuracy: 28/80 (35%)

training epoch 43
Train Epoch: 43 [0/320 (0%)]	Loss: 0.149572
Train Epoch: 43 [40/320 (12%)]	Loss: 0.118291
Train Epoch: 43 [80/320 (25%)]	Loss: 0.151920
Train Epoch: 43 [1

Train Epoch: 60 [160/320 (50%)]	Loss: 0.152598
Train Epoch: 60 [200/320 (62%)]	Loss: 0.124106
Train Epoch: 60 [240/320 (75%)]	Loss: 0.164649
Train Epoch: 60 [280/320 (88%)]	Loss: 0.153917

Test set: Accuracy: 28/80 (35%)

training epoch 61
Train Epoch: 61 [0/320 (0%)]	Loss: 0.126717
Train Epoch: 61 [40/320 (12%)]	Loss: 0.151086
Train Epoch: 61 [80/320 (25%)]	Loss: 0.151003
Train Epoch: 61 [120/320 (38%)]	Loss: 0.171457
Train Epoch: 61 [160/320 (50%)]	Loss: 0.135592
Train Epoch: 61 [200/320 (62%)]	Loss: 0.149931
Train Epoch: 61 [240/320 (75%)]	Loss: 0.155020
Train Epoch: 61 [280/320 (88%)]	Loss: 0.131721

Test set: Accuracy: 28/80 (35%)

training epoch 62
Train Epoch: 62 [0/320 (0%)]	Loss: 0.126469
Train Epoch: 62 [40/320 (12%)]	Loss: 0.135513
Train Epoch: 62 [80/320 (25%)]	Loss: 0.147646
Train Epoch: 62 [120/320 (38%)]	Loss: 0.150674
Train Epoch: 62 [160/320 (50%)]	Loss: 0.134207
Train Epoch: 62 [200/320 (62%)]	Loss: 0.169135
Train Epoch: 62 [240/320 (75%)]	Loss: 0.159851
Train Epoch: 


Test set: Accuracy: 28/80 (35%)

training epoch 80
Train Epoch: 80 [0/320 (0%)]	Loss: 0.138182
Train Epoch: 80 [40/320 (12%)]	Loss: 0.124672
Train Epoch: 80 [80/320 (25%)]	Loss: 0.148509
Train Epoch: 80 [120/320 (38%)]	Loss: 0.171672
Train Epoch: 80 [160/320 (50%)]	Loss: 0.130535
Train Epoch: 80 [200/320 (62%)]	Loss: 0.163291
Train Epoch: 80 [240/320 (75%)]	Loss: 0.153011
Train Epoch: 80 [280/320 (88%)]	Loss: 0.157868

Test set: Accuracy: 28/80 (35%)

training epoch 81
Train Epoch: 81 [0/320 (0%)]	Loss: 0.180838
Train Epoch: 81 [40/320 (12%)]	Loss: 0.200191
Train Epoch: 81 [80/320 (25%)]	Loss: 0.158999
Train Epoch: 81 [120/320 (38%)]	Loss: 0.140685
Train Epoch: 81 [160/320 (50%)]	Loss: 0.149646
Train Epoch: 81 [200/320 (62%)]	Loss: 0.137587
Train Epoch: 81 [240/320 (75%)]	Loss: 0.145252
Train Epoch: 81 [280/320 (88%)]	Loss: 0.154999

Test set: Accuracy: 28/80 (35%)

training epoch 82
Train Epoch: 82 [0/320 (0%)]	Loss: 0.170898
Train Epoch: 82 [40/320 (12%)]	Loss: 0.166021
Train Epoch:

Train Epoch: 99 [160/320 (50%)]	Loss: 0.174757
Train Epoch: 99 [200/320 (62%)]	Loss: 0.141196
Train Epoch: 99 [240/320 (75%)]	Loss: 0.173066
Train Epoch: 99 [280/320 (88%)]	Loss: 0.158185

Test set: Accuracy: 28/80 (35%)



In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
torch.save(model.state_dict(), 'dataset_model_soundemotion.pt')

## NOTES

below is the mfccs notes / random code

In [13]:
audio, sample_rate = librosa.load("SongEmotionDataset/1.mp3", res_type='kaiser_fast')
# [print(x) for x in audio]

#convert audio into 2d array
mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
# mfccsscaled = np.mean(mfccs.T,axis=0)
print(mfccs.shape, audio.shape)
mfccs

/home/czf/memories/venv/lib/python3.6/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(40, 2586) (1323648,)


array([[-5.30341797e+02, -4.07741577e+02, -3.27536621e+02, ...,
        -2.39811523e+02, -1.96744080e+02, -1.44711777e+02],
       [ 5.81265569e-01,  1.03006027e+02,  1.29354553e+02, ...,
         1.48707626e+02,  1.45873001e+02,  1.28202530e+02],
       [ 4.58764762e-01,  7.53921986e+00, -1.18814125e+01, ...,
        -2.51551704e+01, -1.92207527e+01, -1.79366188e+01],
       ...,
       [ 3.11299562e-01, -1.29907084e+00,  1.18818974e+00, ...,
        -6.58579540e+00, -3.34302998e+00, -4.75482178e+00],
       [ 2.23848164e-01, -3.19489312e+00, -2.78556681e+00, ...,
        -1.36089420e+01, -6.40699673e+00, -5.27228928e+00],
       [ 8.67742151e-02,  1.31472754e+00, -1.41885233e+00, ...,
         3.34440261e-01,  1.14392626e+00, -3.62402201e-02]], dtype=float32)

In [10]:
# audio_tensor = torch.tensor(audio)
# audio_tensor
# audio_tensor.shape

In [16]:
# for sound_file in data_path.iterdir():
#     if ".mp3" in str(sound_file):
#         print(sound_file)
#         audio, sample_rate = librosa.load(str(sound_file), res_type='kaiser_fast')
        
    